In [1]:
from selenium.webdriver import ActionChains
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
def combinations(name):
    string=name.upper().split(" ")
    combinations=[" ".join(string[:x+1]) for x in range(0,len(string)) ]
    combinations.reverse()
    return combinations

In [3]:
browser = webdriver.Chrome()
browser.get('https://www.linkedin.com/')
browser.implicitly_wait(5)
elem = browser.find_element_by_name('session_key')
elem.clear()
elem.send_keys("") # your email id 
elem = browser.find_element_by_name('session_password')
elem.clear()
elem.send_keys("") # your password
submit = browser.find_element_by_xpath("//*[@type='submit']")
actions = ActionChains(browser)
actions.click(submit)
actions.perform() 
df=pd.read_csv("companies_name.csv")
companies=df["Company Name"]
companies=companies.dropna()

names=[]
websites=[]
headquarterss=[]
yearfoundeds=[]
companytypes=[]
sizes=[]
specialitys=[]
information=[]


for company in list(companies):
    companies_name=combinations(company)
    for name in companies_name:


        try:    
                browser.get('https://www.linkedin.com/company')
                search=browser.find_element_by_class_name("typeahead__input")
                search.send_keys(name)
                time.sleep(3)
                if browser.find_element_by_xpath("/html/body/main/section[1]/section/div/ul/li[1]/a"):
                    print(company)
                    elem=browser.find_element_by_xpath("/html/body/main/section[1]/section/div/ul/li[1]/a")
                    elem.click()
                    time.sleep(2) 
                    soup = BeautifulSoup(browser.page_source, 'lxml')
                    text = soup.text.replace("\n", "$")

                    text = text.replace("  ", "")
                    text = text.replace("$$", "$")
                    string = text[text.find("Company details") + len("Company details") + 1: text.find("See more details about")]
                    string = string.split("$")
                    for itr in range(0, len(string)):
                        try:
                            string.remove(" ")
                        except:
                              pass
                        try:
                            string.remove(" ")
                        except:
                              pass
                        try:
                            string.remove("")
                        except:
                              pass
                    
                    dictionary = {}
                    for itr in range(0, len(string) - 1, 2):
                        dictionary[string[itr]] = string[itr + 1]
                    data = []
                    data.append(company)
                    data.append(name)
                    main = {
                      "Website": 0,
                      "Headquarters": 0,
                      "Year founded": 0,
                      "Company type": 0,
                      "Company size": 0,
                      "Specialties": 0
                    }
                    for key in dictionary.keys():
                        if key in main:
                              main[key] = dictionary[key]
                    for value in main.values():
                        data.append(value)

                    information.append(data)
                    break
        except:
            pass

In [4]:
pd.DataFrame(information).to_csv("companies_data_linked.csv",index=False)